In [1]:
import re
import pandas as pd
from typing import Tuple, Optional, Union
from IPython import get_ipython

In [ ]:
"""

READ DATA AND PREPARE ALL

"""

In [72]:
path = get_ipython().run_line_magic('pwd','')
print(path)

### EDIT MANUALLY
path1 = r"C:\Users\cacr4002\notebooks\Coverage AI\Text-NLP\Data\Raw\niq_products_data.csv"
path2 = r"C:\Users\cacr4002\notebooks\Coverage AI\Text-NLP\Data\Raw\niq_products_asia_europe.csv"

#print(os.path.exists(paths))

C:\Users\cacr4002\notebooks\Coverage AI\Text-NLP


In [73]:
### EDIT MANUALLY
### RUN ONCE

DF_NIQ1 = pd.read_csv(path1) 
DF_NIQ2 = pd.read_csv(path2, encoding='cp1252')

In [74]:
# Define cols to filter by
cols1 = ['Product Name','Tamanio','Unidades']
cols2 = ['Product','Tamanio','Unidades']
### FILTER
DF_NIQ1 = DF_NIQ1[cols1]
DF_NIQ2 = DF_NIQ2[cols2]

dfs = []
for df in [DF_NIQ1,DF_NIQ2]:
    df_renamed = df.copy()
    first_col = df.columns[0]
    df_renamed = df_renamed.rename(columns={first_col: 'Product Name'})
    dfs.append(df_renamed)

# Paso 2: Unir todos los DataFrames
DF_NIQ_FINAL = pd.concat(dfs, ignore_index=True)

In [75]:
DF_NIQ_FINAL.head()

,Product Name,Tamanio,Unidades
0,AMERICANO GANCIA 14Â° 12x950,950.0,ml
1,AMERICANO GANCIA 14Âº 12x 450 ml,450.0,ml
2,AMERICANO GANCIA 14Â° 6 x1250ML,1250.0,ml
3,AMARGO OBRERO 19Â° 12x950,950.0,ml
4,GANCIA HIBISCUS SPRITZ 25Â° 6X750ML,750.0,ml


In [76]:
DF_NIQ_FINAL['Product Name'].size

1901

In [77]:
DF_NIQ_FINAL['Unidades'].value_counts()

Unidades
ml       744
g        548
G        387
l         45
units     36
kg        36
oz        28
P         23
ML        19
L         17
GR         3
KG         1
p          1
Units      1
Name: count, dtype: int64

In [78]:
"""

DELETE NULLS AND UNIFY UNITS OF MEASUREMENT

"""

'\n\nDELETE NULLS AND UNIFY UNITS OF MEASUREMENT\n\n'

In [79]:
DF_NIQ_FINAL = DF_NIQ_FINAL.dropna(subset=['Tamanio'])
DF_NIQ_FINAL = DF_NIQ_FINAL.dropna(subset=['Unidades'])

In [80]:
DF_NIQ_FINAL['Unidades'] = DF_NIQ_FINAL['Unidades'].str.lower()
DF_NIQ_FINAL['Unidades'] = DF_NIQ_FINAL['Unidades'].str.replace("gr","g")
DF_NIQ_FINAL = DF_NIQ_FINAL[DF_NIQ_FINAL['Unidades'] != 'p']

In [81]:
DF_NIQ_FINAL['Product Name'].size

1864

In [82]:
DF_NIQ_FINAL['Unidades'].value_counts()

Unidades
g        938
ml       763
l         62
kg        37
units     36
oz        28
Name: count, dtype: int64

In [83]:
conversion_facts = {
    'kg':1000,
    'l':1000,
    'oz':29.57
}
def convert_units(row):
    unit = row['Unidades']
    value = row['Tamanio']
    if unit in conversion_facts:
        if unit == 'kg':
            new_unit = 'g'
        elif unit == 'l' or unit == 'oz':
            new_unit = 'ml'
        else:
            new_unit = unit
        return pd.Series([value* conversion_facts[unit], new_unit])
    return pd.Series([value,unit])

    
DF_NIQ_FINAL[['Tamaño', 'Unidades']] = DF_NIQ_FINAL.apply(convert_units, axis=1)

In [84]:
DF_NIQ_FINAL['Unidades'].value_counts()

Unidades
g        975
ml       853
units     36
Name: count, dtype: int64

In [85]:
"""

SEND ALL TO LOWER CASE AND DELETE INFORMATION FROM PRODUCT THAT HAS [] OR ()

"""

'\n\nSEND ALL TO LOWER CASE AND DELETE INFORMATION FROM PRODUCT THAT HAS [] OR ()\n\n'

In [86]:
DF_NIQ_FINAL['Product Name'] = DF_NIQ_FINAL['Product Name'].str.replace(r"[\[\(].*?[\]\)]","",regex=True)

In [87]:
"""

NLP STARTS

"""

'\n\nNLP STARTS\n\n'

In [88]:
# -----------------------------
# Configuración y normalización
# -----------------------------

"""
Definir un diccionario de unidades porque no siempre están bien escritas o en el mismo formato. Se exponen las formas literales de hacerlo. 
* Si hay una nueva forma, debe incluirse a mano.
"""

# NOTA: Se esperan unidades en minúscula
ALIAS_TO_CANONICAL = {
    # Volumen directo a ml
    "ml": ("ml", 1),
    "cc": ("ml", 1), "cm3": ("ml", 1), "cm":("ml", 1),
    "cl": ("ml", 10),
    "l": ("ml", 1000), "lt": ("ml", 1000), "lts": ("ml", 1000),
    "litro": ("ml", 1000), "litros": ("ml", 1000),
    # Inglés / variantes
    "milliliter": ("ml", 1), "milliliters": ("ml", 1),
    "millilitre": ("ml", 1), "millilitres": ("ml", 1),
    "mililiter": ("ml", 1), "mililiters": ("ml", 1),  # por si hay typos comunes
    "millilitro": ("ml", 1), "millilitros": ("ml", 1),
    "mililitro": ("ml", 1), "mililitros": ("ml", 1),
    "liter": ("ml", 1000), "liters": ("ml", 1000),
    "litre": ("ml", 1000), "litres": ("ml", 1000),

    # Masa directo a gr
    "g": ("gr", 1), "gr": ("gr", 1), "grs": ("gr", 1),
    "gm": ("gr", 1), "gms": ("gr", 1),
    "gram": ("gr", 1), "grams": ("gr", 1),
    "gramo": ("gr", 1), "gramos": ("gr", 1),
    "kg": ("gr", 1000), "kgs": ("gr", 1000),
    "kilo": ("gr", 1000), "kilos": ("gr", 1000),
    "k": ("gr", 1000),  # p.ej., "3k" => 3000 gr

    # Otras (opcional)
    "oz": ("ml", 29.57),  # aprox. a ml
}

# Lista de tamaños volumétricos típicos (ml) para inferencia neutral (multilenguaje)
"""
Esto es porque muchas veces hay muchos numeros en el nombre del producto que no aportan, entonces se priorizará solo revisar numeros que están en este listado.
* Si hay un tamaño de mucha frecuencia, puede incluirse también.
"""
COMMON_VOLUME_SIZES = {
    187, 200, 237, 250, 270, 275, 300, 310, 312, 320, 330, 340, 350, 355, 375,
    400, 410, 420, 440, 450, 473, 480, 500, 550, 568, 590, 600, 620, 650, 660,
    680, 700, 710, 720, 740, 750, 770, 800, 850, 900, 940, 950, 970, 990, 1000,
    1125, 1180, 1200, 1250, 1500, 1750, 2000, 2250, 2500, 2700, 3000, 3500, 4000, 5000
}

"""
Palabras genéricas de contenedor (multilenguaje) para inferir volumen cuando no hay unidad explícita. Es decir: Lata 700
* Debe ser modificado a mano para nuevos casos
"""
CONTAINER_HINTS = {
    "can", "tin", "lata", "bottle", "botella", "bouteille", "flasche", "bottiglia",
    "jar", "frasco", "pouch", "sachet", "pack", "box", "caja", "estuche", "carton",
    "display", "bundle", "tray", "keg", "brick", "tetra", "tetrapak", "tetra-pack",
    "tube", "tubo", "ampoule", "ampolla", "vial", "bag", "bolsa"
}

def _canon_from_alias(unit_text: str):
    u = unit_text.strip().lower().replace(' ', '')
    u = u.rstrip('.')
    return ALIAS_TO_CANONICAL.get(u, None)

def _num_to_str(val: Union[int, float]) -> str:
    """Render numérico estable: enteros sin .0, decimales sin ceros de cola."""
    if isinstance(val, int):
        return str(val)
    try:
        f = float(val)
        if abs(f - round(f)) < 1e-6:
            return str(int(round(f)))
        s = f"{f:.6f}".rstrip('0').rstrip('.')
        return s
    except Exception:
        return str(val)

def _to_canonical(num_str: str, unit_str: str) -> Tuple[Union[int, float], str]:
    """Convierte número+unidad a (valor, unidad_canónica) con conversiones (kg->gr, l->ml, etc.)."""
    num = float(num_str.replace(',', '.'))
    m = _canon_from_alias(unit_str)
    if not m:
        return num, "XXX"
    canonical_unit, factor = m
    val = num * factor
    if abs(val - round(val)) < 1e-6:
        val = int(round(val))
    return val, canonical_unit

def _has_container_hint(text_lower: str) -> bool:
    """True si hay palabras de envase/packaging."""
    words = set(re.findall(r'[a-zA-Záéíóúñüäöëïç\-]+', text_lower))
    words = {w.lower() for w in words}
    return len(words.intersection(CONTAINER_HINTS)) > 0

def _is_common_volume(n_str: str) -> bool:
    """True si hay un tamaño muy frecuente"""
    try:
        f = float(n_str.replace(',', '.'))
    except Exception:
        return False
    # Para enteros, revisar en lista
    if abs(f - round(f)) < 1e-6:
        return int(round(f)) in COMMON_VOLUME_SIZES
    return False

def _looks_like_volume(n_str: str) -> bool:
    """Heurística amplia: ¿parece volumen en ml? Acepta decimales."""
    try:
        # Formato estandar de cómo escribir un decimal
        f = float(n_str.replace(',', '.')) 
    except Exception:
        return False
    # Enteros típicos o múltiplos de 50/100 dentro de rango
    if abs(f - round(f)) < 1e-6:
        ni = int(round(f))
        if ni in COMMON_VOLUME_SIZES:
            return True
        return 180 <= ni <= 5000 and (ni % 100 == 0 or ni % 50 == 0)
    # Decimales plausibles para volúmenes pequeños/medianos
    return 5.0 <= f <= 5000.0

def _is_plausible_packcount(n_str: str) -> bool:
    """Conteos de pack típicos. Se puede ajustar manual el rango pero entre mas alto, mayor probabilidad de errores"""
    try:
        n = int(float(n_str.replace(',', '.')))
    except Exception:
        return False
    return 2 <= n <= 72  # packs habituales (puedes subir a 144 si tu dominio lo requiere)

# -----------------------------
# Regex
# -----------------------------

""" ESTRATEGIA
Primero intentamos encontrar número + unidad (junto o separado).
Si no, probamos pack A x B o A / B (con o sin unidad).
Si hay unidad → usar ese número como tamaño.
Si no hay → priorizar si uno está en COMMON_VOLUME_SIZES (p. ej., 950).
Si no, buscamos patrones de conteo de unidades (6u/12, 24/Caja, ESTUCHES 24).
Si nada de lo anterior, capturamos un número suelto y decidimos si parece volumen (ml) o dejarlo como XXX.
"""

# UNIT_RE generado dinámicamente de alias (ordenado por longitud desc para evitar "l" dentro de "liter")
_UNIT_ALIASES = sorted(ALIAS_TO_CANONICAL.keys(), key=len, reverse=True) # Ordernar llaves de ALIAS_TO_CANONICAL
UNIT_RE = r'(?:' + '|'.join(re.escape(u) for u in _UNIT_ALIASES) + r')' # Definición de las unidades de un producto
# Números: enteros o decimales con . o ,
NUM_RE = r'(\d+(?:[.,]\d+)?)'
NUM_RE_NOGRP = r'\d+(?:[.,]\d+)?'
# Al terminar unidad, permitir fin, no-letra, o seguidor de pack (x, ×, /)
UNIT_FOLLOW = r'(?=$|[^a-zA-Z]|[xX×/])'

# 1) "número + unidad" (separado o pegado), soporta '900gx18u', '800mlx14'
PAT_NUM_UNIT_SEP = re.compile(rf'{NUM_RE}\s*({UNIT_RE}){UNIT_FOLLOW}', re.I)
PAT_NUM_UNIT_FUSED = re.compile(rf'{NUM_RE}({UNIT_RE}){UNIT_FOLLOW}', re.I)

# 2) "pack" tipo "12x 450 ml" o "12/950" o "6/32.9" (acepta ×)
PAT_PACK = re.compile(rf'(\d+)\s*[*xX×/]\s*({NUM_RE_NOGRP})(?:\s*({UNIT_RE}){UNIT_FOLLOW})?', re.I)

# 3) "unidades" (multilenguaje)
UNITS_TOK = r'(?:u|un|ud|uds|und\.?|und|unid(?:ad(?:es)?)?|units?|pcs?|pz|pieza(?:s)?)' #Definir formas de encontrar las unidades. Cambiar a mano si es necesario.
# Aqui se definen los casos "12un/24u" o "6u/12"
PAT_UNITS_BOTH = re.compile(rf'(\d+)\s*{UNITS_TOK}\s*/\s*(\d+)\s*{UNITS_TOK}\b', re.I)
PAT_UNITS_FIRST = re.compile(rf'(\d+)\s*{UNITS_TOK}\s*/\s*(\d+)\b', re.I)
# Para cosas como "24/Caja" o "12/box".
PAT_SLASH_CAJA = re.compile(r'(\d+)\s*/\s*(?:caja|cj|c\/j|box|case)\b', re.I)
PAT_WORD_NUM = re.compile(r'(?:estuche(?:s)?|pack|paquete(?:s)?|caja(?:s)?|box(?:es)?|blister(?:s)?|display(?:s)?|case(?:s)?)\D*(\d+)\b', re.I) # Editar si cambia.

# 4) Números sueltos (incluye decimales)
PAT_ANY_NUM = re.compile(rf'\b{NUM_RE_NOGRP}\b')

In [89]:
def extract_size_unit(product_name: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Regresa (size, unit) donde:
      - unit in {'ml', 'gr', 'units', 'XXX'}
      - size es string (permite '12/24' y decimales como '32.9')
    Reglas clave:
      * Prioriza coincidencias explícitas "número+unidad" (soporta palabras de unidad).
      * En paquetes A/B o AxB sin unidad, prioriza el lado que está en COMMON_VOLUME_SIZES.
      * Si ninguno está en la lista, elige el lado que "parece volumen" y el otro parece conteo de pack.
      * Evita inventar: si no hay señales, devuelve 'XXX'.
    """
    if not product_name or not isinstance(product_name, str):
        return None, None

    s = product_name.strip()
    s_clean = s.lower()
    # Normalizaciones de encoding y variantes
    s_clean = (s_clean
               .replace('c.c.', 'cc')
               .replace('c.c', 'cc')
               .replace('Â°', '°')  # correcciones comunes de encoding
               .replace('â°', '°'))

    # 1) "número + unidad" (prioritario) — soporta decimales y unidad pegada seguida de x/ /
    m = PAT_NUM_UNIT_SEP.search(s_clean) or PAT_NUM_UNIT_FUSED.search(s_clean)
    if m:
        num, unit = m.group(1), m.group(2)
        val, canonical = _to_canonical(num, unit)
        return (_num_to_str(val), canonical)

    # 2) Paquetes "A/B" o "AxB"
    mp = PAT_PACK.search(s_clean)
    if mp:
        left, right, unit = mp.groups()
        if unit:
            # Caso "12x 450 ml"
            val, canonical = _to_canonical(right, unit)
            return (_num_to_str(val), canonical)
        else:
            # Sin unidad explícita: priorizar COMMON_VOLUME_SIZES
            left_common = _is_common_volume(left)
            right_common = _is_common_volume(right)
            if right_common and not left_common:
                return (_num_to_str(float(right.replace(',', '.'))), 'ml')
            if left_common and not right_common:
                return (_num_to_str(float(left.replace(',', '.'))), 'ml')
            if left_common and right_common:
                # Heurística: usualmente el tamaño va a la derecha (12/750 => 750 ml)
                return (_num_to_str(float(right.replace(',', '.'))), 'ml')

            # Si ninguno está en la lista, usar heurística volumen vs. pack
            left_vol = _looks_like_volume(left)
            right_vol = _looks_like_volume(right)
            left_pack = _is_plausible_packcount(left)
            right_pack = _is_plausible_packcount(right)

            # Preferir el que parezca volumen y el otro conteo de pack
            if right_vol and left_pack and not left_vol:
                return (_num_to_str(float(right.replace(',', '.'))), 'ml')
            if left_vol and right_pack and not right_vol:
                return (_num_to_str(float(left.replace(',', '.'))), 'ml')

            # Como señal adicional, si hay contenedor + uno parece volumen, tomarlo
            if _has_container_hint(s_clean):
                if right_vol:
                    return (_num_to_str(float(right.replace(',', '.'))), 'ml')
                if left_vol:
                    return (_num_to_str(float(left.replace(',', '.'))), 'ml')
            # Sin inferencias sólidas -> continuar

    # 3) Conteos de unidades (cuando no hay volumen/masa explícito)
    mu_both = PAT_UNITS_BOTH.search(s_clean)
    if mu_both:
        n1, n2 = mu_both.group(1), mu_both.group(2)
        return (f"{int(n1)}", "units")

    mu_first = PAT_UNITS_FIRST.search(s_clean)
    if mu_first:
        n1 = mu_first.group(1)
        return (str(int(n1)), "units")

    mcj = PAT_SLASH_CAJA.search(s_clean)
    if mcj:
        return (str(int(mcj.group(1))), "units")

    mpw = PAT_WORD_NUM.search(s_clean)
    if mpw:
        return (str(int(mpw.group(1))), "units")

    # 4) Números sueltos: inferir ml sólo si hay contenedor o tamaño típico/plausible
    anyn = PAT_ANY_NUM.search(s_clean)
    if anyn:
        raw = anyn.group(0)
        if _is_common_volume(raw) or (_has_container_hint(s_clean) and _looks_like_volume(raw)):
            return (_num_to_str(float(raw.replace(',', '.'))), 'ml')
        # Fallback seguro: no inventar unidad
        return (_num_to_str(float(raw.replace(',', '.'))), "XXX")

    return None, None

In [90]:
"""

Definir funcion para probar el modelo

"""

'\n\nDefinir funcion para probar el modelo\n\n'

In [91]:
def _normalize_number_like(s: Optional[str]) -> Optional[str]:
    """Normaliza strings numéricos para comparación: maneja ',' como '.', recorta ceros."""
    if s is None:
        return None
    st = str(s).strip().lower().replace(',', '.')
    try:
        f = float(st)
        if abs(f - round(f)) < 1e-9:
            return str(int(round(f)))
        return f"{f:.6f}".rstrip('0').rstrip('.')
    except Exception:
        return st

def normalize_expected_value(v) -> Optional[str]:
    return _normalize_number_like(v)

def normalize_expected_unit(u) -> Optional[str]:
    if u is None:
        return None
    u = str(u).strip().lower()
    m = _canon_from_alias(u)
    if m:
        can, _ = m
        return can
    if u in {"unit", "units", "un", "u", "ud", "uds", "und", "und.", "unid", "unids", "unidades", "pcs", "pz", "pieza", "piezas"}:
        return "units"
    if u == "xxx":
        return "xxx"
    return u

def evaluate_extraction(df: pd.DataFrame,
                        product_col: str = "Product Name",
                        size_col: str = "Tamaño",
                        unit_col: str = "Unidades") -> pd.DataFrame:
    """
    Aplica extractor y calcula métricas:
      - accuracy size
      - accuracy unit
      - accuracy ambas
    Devuelve un DataFrame con predicciones y flags de acierto, y hace un resumen por print.
    """
    df = df.copy()
    preds = df[product_col].apply(extract_size_unit)
    df["pred_size"] = preds.apply(lambda x: x[0])
    df["pred_unit"] = preds.apply(lambda x: x[1])

    df["_size_true"] = df[size_col].apply(normalize_expected_value)
    df["_unit_true"] = df[unit_col].apply(normalize_expected_unit)
    df["_size_pred"] = df["pred_size"].apply(normalize_expected_value)
    df["_unit_pred"] = df["pred_unit"].apply(normalize_expected_unit)

    df["size_ok"] = df["_size_pred"] == df["_size_true"]
    df["unit_ok"] = df["_unit_pred"] == df["_unit_true"]
    df["both_ok"] = df["size_ok"] & df["unit_ok"]

    size_acc = df["size_ok"].mean()
    unit_acc = df["unit_ok"].mean()
    both_acc = df["both_ok"].mean()

    print("=== Métricas de extracción ===")
    print(f"Accuracy size: {size_acc:.3f}")
    print(f"Accuracy unit: {unit_acc:.3f}")
    print(f"Accuracy ambos: {both_acc:.3f}")

    unit_summary = (df
        .groupby("_unit_true", dropna=False)
        .agg(
            n=("Product Name", "count"),
            size_acc=("size_ok", "mean"),
            unit_acc=("unit_ok", "mean"),
            both_acc=("both_ok", "mean")
        )
        .sort_values("n", ascending=False)
    )
    print("\n=== Resumen por unidad esperada ===")
    print(unit_summary.to_string())

    return df[[
        product_col, size_col, unit_col, "pred_size", "pred_unit",
        "size_ok", "unit_ok", "both_ok"
    ]]

In [92]:
results = evaluate_extraction(DF_NIQ_FINAL)
print("\n=== Predicciones detalle ===")
print(results.to_string(index=False))

=== Métricas de extracción ===
Accuracy size: 0.984
Accuracy unit: 0.983
Accuracy ambos: 0.981

=== Resumen por unidad esperada ===
              n  size_acc  unit_acc  both_acc
_unit_true                                   
gr          975  0.991795  0.989744  0.988718
ml          853  0.989449  0.988277  0.987104
units        36  0.666667  0.694444  0.638889

=== Predicciones detalle ===
                                    Product Name    Tamaño Unidades pred_size pred_unit  size_ok  unit_ok  both_ok
                   AMERICANO GANCIA 14Â°  12x950   950.000       ml       950        ml     True     True     True
                AMERICANO GANCIA 14Âº 12x 450 ml   450.000       ml       450        ml     True     True     True
                AMERICANO GANCIA 14Â°  6 x1250ML  1250.000       ml      1250        ml     True     True     True
                      AMARGO OBRERO 19Â°  12x950   950.000       ml       950        ml     True     True     True
             GANCIA HIBISCUS SPRI